In [1]:
using Revise
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.resolve()

include("../src/ChemQHam.jl")
using .ChemQHam

# using ITensorChemistry
# using SparseArrays
# using LinearAlgebra


  Activating project at `~/Desktop/Felman/projects/Quantum/ChemQHam`
  No Changes to `~/Desktop/Felman/projects/Quantum/ChemQHam/Project.toml`
  No Changes to `~/Desktop/Felman/projects/Quantum/ChemQHam/Manifest.toml`


In [5]:
using SparseArrays
using LinearAlgebra

In [2]:
# ITensorChemistry opsum generation
molecule = Molecule([("Li", 0.00, 0.00, 0.0000), ("H", 0.00, 0.00, 1.000)])
(; opsum, h1e, h2e, nuc_e) = ITChem_opsum(molecule)
println("opsum: ", opsum)


# TensorKit opsum generation
op_terms = gen_ChemOpSum(h1e, h2e, nuc_e)
println("op_terms: ", op_terms)


opsum: sum(
  1.5875316327600002 Id(1,)
  -4.921360416540289 c†↑(1,) c↑(1,)
  -4.921360416540289 c†↓(1,) c↓(1,)
  0.14792631902885287 c†↑(1,) c↑(2,)
  0.14792631902885287 c†↓(1,) c↓(2,)
  0.17076042394105295 c†↑(1,) c↑(3,)
  0.17076042394105295 c†↓(1,) c↓(3,)
  0.07075423762109773 c†↑(1,) c↑(6,)
  0.07075423762109773 c†↓(1,) c↓(6,)
  0.14792631902885287 c†↑(2,) c↑(1,)
  0.14792631902885287 c†↓(2,) c↓(1,)
  -1.7459768632351027 c†↑(2,) c↑(2,)
  -1.7459768632351027 c†↓(2,) c↓(2,)
  0.04857005588863113 c†↑(2,) c↑(3,)
  0.04857005588863113 c†↓(2,) c↓(3,)
  -0.32648453610749234 c†↑(2,) c↑(6,)
  -0.32648453610749234 c†↓(2,) c↓(6,)
  0.1707604239410531 c†↑(3,) c↑(1,)
  0.1707604239410531 c†↓(3,) c↓(1,)
  0.048570055888631135 c†↑(3,) c↑(2,)
  0.048570055888631135 c†↓(3,) c↓(2,)
  -1.1757051929723854 c†↑(3,) c↑(3,)
  -1.1757051929723854 c†↓(3,) c↓(3,)
  0.03525707086814607 c†↑(3,) c↑(6,)
  0.03525707086814607 c†↓(3,) c↓(6,)
  -1.1981645481561318 c†↑(4,) c↑(4,)
  -1.1981645481561318 c†↓(4,) c↓(4,

In [53]:
# TensorKit opsum to TensorKit MPO and sparse mat
table, primary_ops, factors = terms_to_table(op_terms)
symbolic_mpo, virt_spaces = construct_symbolic_mpo(table, primary_ops, factors; verbose=false)
mpo = symbolic_to_tensorkit_mpo(symbolic_mpo, virt_spaces, "U1SU2"; verbose=false)
tk_mat = mpo_to_mat(mpo)
tk_nz = collect(zip(findnz(tk_mat)...))

# ITensorChemistry opsum to sparse mat
it_mat = ITChem_mat(opsum)
it_nz = collect(zip(findnz(it_mat)...))



┌ Warning: Operator c2c1 with left QN (0, 0, 0//1) and right QN (0, 2, 1//1) not found in the dictionary.
└ @ Main.ChemQHam /home/daniel/Desktop/Felman/projects/Quantum/ChemQHam/src/tensorkit_utils.jl:715
┌ Warning: Operator a1a2 with left QN (0, 0, 0//1) and right QN (0, -2, 1//1) not found in the dictionary.
└ @ Main.ChemQHam /home/daniel/Desktop/Felman/projects/Quantum/ChemQHam/src/tensorkit_utils.jl:715
┌ Warning: Operator c2c1 with left QN (0, 0, 0//1) and right QN (0, 2, 1//1) not found in the dictionary.
└ @ Main.ChemQHam /home/daniel/Desktop/Felman/projects/Quantum/ChemQHam/src/tensorkit_utils.jl:715
┌ Warning: Operator a1a2 with left QN (0, 0, 0//1) and right QN (0, -2, 1//1) not found in the dictionary.
└ @ Main.ChemQHam /home/daniel/Desktop/Felman/projects/Quantum/ChemQHam/src/tensorkit_utils.jl:715
┌ Warning: Operator I with left QN (0, 2, 0//1) and right QN (0, 2, 1//1) not found in the dictionary.
└ @ Main.ChemQHam /home/daniel/Desktop/Felman/projects/Quantum/ChemQHam/src

143104-element Vector{Tuple{Int64, Int64, Float64}}:
 (1, 1, 1.5875316327600002)
 (2, 2, -3.333828783780289)
 (5, 2, 0.14792631902885295)
 (17, 2, 0.170760423941053)
 (1025, 2, 0.07075423762109787)
 (3, 3, -3.333828783780289)
 (9, 3, 0.14792631902885287)
 (33, 3, 0.17076042394105287)
 (2049, 3, 0.07075423762109763)
 (4, 4, -6.60978477113754)
 ⋮
 (2048, 4094, 0.0015657933738627595)
 (4064, 4094, 0.06542842150969506)
 (4088, 4094, 0.052548005836265475)
 (4094, 4094, 1.1431675314795848)
 (3072, 4095, 0.0015657933738627379)
 (4080, 4095, 0.06542842150969508)
 (4092, 4095, 0.052548005836265455)
 (4095, 4095, 1.1431675314795848)
 (4096, 4096, 1.8660223467508983)

In [54]:
# compare
norm_diff = norm(tk_mat-it_mat)
println("norm(tk_mat - it_mat) = ", norm_diff)

norm(tk_mat - it_mat) = 1.768541435661743e-12
